<a href="https://colab.research.google.com/github/Vibha111094/OCR/blob/main/OCR_fun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytesseract
!pip install opencv-python
!pip install opencv_wrapper
!pip install colorama

  Created wheel for pytesseract: filename=pytesseract-0.3.6-py2.py3-none-any.whl size=13629 sha256=d433b881a6e5e8e81af43a37bbeda9910a06ddc338bbfae3998a56c1bcc95239
  Stored in directory: /root/.cache/pip/wheels/ee/71/72/b98430261d849ae631e283dfc7ccb456a3fb2ed2205714b63f
Successfully built pytesseract
     |████████████████████████████████| 17.3MB 342kB/s 
     |████████████████████████████████| 25.4MB 142kB/s 
  Created wheel for opencv-wrapper: filename=opencv_wrapper-0.2.3-py2.py3-none-any.whl size=18580 sha256=fafeaf4f78a77f273c273edb60b4278d2e6bdc21eeaa93b83b47c8c69965a680
  Stored in directory: /root/.cache/pip/wheels/74/fc/22/03106b3bcb2516e69317562a2aa4ceb94b0d5813a448edd928
Successfully built opencv-wrapper
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.16.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0

In [1]:
!sudo apt-get update
!sudo apt-get install libleptonica-dev 
!sudo apt-get install tesseract-ocr tesseract-ocr-dev
!sudo apt-get install libtesseract-dev

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1,814 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu180

In [11]:
!sudo apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 36 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 0s (30.7 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5

In [12]:
!pip install tesseract
!pip install tesseract-ocr

In [13]:
import pytesseract
import cv2

In [4]:
# Import PyTorch Library
import torch
from torch import nn

# Import external libraries
import argparse
import numpy as np
import opencv_wrapper as cvw
from skimage.filters import threshold_local
import json
import random
from string import ascii_uppercase, digits, punctuation
import colorama
import regex

In [5]:
image = cv2.imread('invoice.png')

In [6]:
def get_info(path):
    font     = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.5
    fontColor  = (255,0,0)
    lineType = 1

    #Threshold
    image = cv2.imread(path)

    height,width,channel = image.shape
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    T = threshold_local(gray, 15, offset = 6, method = "gaussian") # generic, mean, median, gaussian
    thresh = (gray > T).astype("uint8") * 255
    thresh = ~thresh

    #Dilation
    kernel =np.ones((1,1), np.uint8)
    ero = cv2.erode(thresh, kernel, iterations= 1)
    img_dilation = cv2.dilate(ero, kernel, iterations=1)

    # Remove noise
    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(img_dilation, None, None, None, 8, cv2.CV_32S)
    sizes = stats[1:, -1] #get CC_STAT_AREA component
    final = np.zeros((labels.shape), np.uint8)
    for i in range(0, nlabels - 1):
        if sizes[i] >= 10:   #filter small dotted regions
            final[labels == i + 1] = 255

    #Find contours
    kern = np.ones((5,15), np.uint8)
    img_dilation = cv2.dilate(final, kern, iterations = 1)
    contours, hierarchy = cv2.findContours(img_dilation, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    # Map contours to bounding rectangles, using bounding_rect property
    rects = map(lambda c: cv2.boundingRect(c), contours)
    # Sort rects by top-left x (rect.x == rect.tl.x)
    sorted_rects = sorted(rects, key =lambda r: r[0])
    sorted_rects = sorted(sorted_rects, key =lambda r: r[1])

    etfo=''
    for rect in sorted_rects:
        x,y,w,h = rect
        if(w<20 or h<20):
            continue
        temp = image[y:y+h, x:x+w]
        temp = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
        hi = pytesseract.image_to_data(temp, config=r'--psm 6')
        hi = hi.split()
        ind = 22
        while(True):
            if (ind>len(hi)):
                break
            if(int(hi[ind])==-1):
                ind+=11
            else:
                etfo=etfo+hi[ind+1]
                etfo=etfo+" "
                x+=len(hi[ind+1])*20
                ind+=12
        etfo=etfo+'\n'
    return etfo

In [7]:
class ExtractLSTM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=2, bidirectional=True)
        self.linear = nn.Linear(hidden_size * 2, 5)

    def forward(self, inpt):
        embedded = self.embed(inpt)
        feature, _ = self.lstm(embedded)
        oupt = self.linear(feature)
        return oupt

In [14]:
etfo = get_info('reciept3.jpg')

In [15]:
etfo

'AROMA CAFE 1211 Green Street New York, NY 05-12-2016 1:11PM TRL 1 HOST ALISON VISA avsorsa2e1 ary _oesc axe 1 ORE GaRT SG 1 ROUSE sauao 08 1 SURE AND TURF — 1 Pens sa: 1 Wine “CLASS - Fixe $50 1 CHOC CAKE $6.75 AMOUNT $90.52 ‘SUB-TOTAL $81.52 Tax $9.00 ‘BALANCE $90.52 \nAROMA CAFE 1211 Green Street New York, NY \n1s1PM HOST ALISON \nee 7:69 48.79 $11.50 $6.75 \n$90.52 \n$81.52 $9.00 $90.52 \nNNN AT A \n'

In [32]:
def pred_to_dict(text, pred, prob):
    res = {"company": ("", 0), "date": ("", 0), "address": ("", 0), "total": ("", 0)}
    keys = list(res.keys())
    seps = [0] + (np.nonzero(np.diff(pred))[0] + 1).tolist() + [len(pred)]
    for i in range(len(seps) - 1):
        pred_class = pred[seps[i]] - 1
        if pred_class == -1:
            continue

        new_key = keys[pred_class]
        new_prob = prob[seps[i] : seps[i + 1]].max()
        if new_prob > res[new_key][1]:
            res[new_key] = (text[seps[i] : seps[i + 1]], new_prob)

    return {k: regex.sub(r"[\t\n]", " ", v[0].strip()) for k, v in res.items()}

 
def test(model):
    model.eval() 
    with torch.no_grad():
            oupt = model(text_tensor)
            prob = torch.nn.functional.softmax(oupt, dim=2)
            prob, pred = torch.max(prob, dim=2)
            prob = prob.squeeze().cpu().numpy()
            pred = pred.squeeze().cpu().numpy()
            real_text = etfo
            result = pred_to_dict(real_text, pred, prob)
            with open("output.json", "w", encoding="utf-8") as json_opened:
                json.dump(result, json_opened, indent=4)
            return result

In [39]:
VOCAB= ascii_uppercase+digits+punctuation+" \t\n"

#Change to CUDA to run using GPU
device = 'cpu'

def get_test_data(etfo):  
    text = etfo
    text_tensor = torch.zeros(len(text), 1, dtype=torch.long)
    text_tensor[:, 0] = torch.LongTensor([VOCAB.find(c) for c in text])
    return text_tensor.to(device)

etfo = get_info('/content/reciept3.jpg')
print(etfo)
#etfo = get_info('X51005621482.jpeg')
etfo = etfo.upper()
text_tensor = get_test_data(etfo)
print(text_tensor)
temp = []
for i in range(len(text_tensor)):
    if text_tensor[i]>=0 and text_tensor[i]<=70:
        temp.append([int(text_tensor[i])])
text_tensor = torch.LongTensor(temp)
temp = []

# #model initialization
hidden_size = 256
device= torch.device('cpu')
model = ExtractLSTM(len(VOCAB), 16, hidden_size).to(device)
model.load_state_dict(torch.load('/content/model.pth'))
result = test(model)
print(result)

AROMA CAFE 1211 Green Street New York, NY 05-12-2016 1:11PM TRL 1 HOST ALISON VISA avsorsa2e1 ary _oesc axe 1 ORE GaRT SG 1 ROUSE sauao 08 1 SURE AND TURF — 1 Pens sa: 1 Wine “CLASS - Fixe $50 1 CHOC CAKE $6.75 AMOUNT $90.52 ‘SUB-TOTAL $81.52 Tax $9.00 ‘BALANCE $90.52 
AROMA CAFE 1211 Green Street New York, NY 
1s1PM HOST ALISON 
ee 7:69 48.79 $11.50 $6.75 
$90.52 
$81.52 $9.00 $90.52 
NNN AT A 

tensor([[ 0],
        [17],
        [14],
        [12],
        [ 0],
        [68],
        [ 2],
        [ 0],
        [ 5],
        [ 4],
        [68],
        [27],
        [28],
        [27],
        [27],
        [68],
        [ 6],
        [17],
        [ 4],
        [ 4],
        [13],
        [68],
        [18],
        [19],
        [17],
        [ 4],
        [ 4],
        [19],
        [68],
        [13],
        [ 4],
        [22],
        [68],
        [24],
        [14],
        [17],
        [10],
        [47],
        [68],
        [13],
        [24],
        [68],
        [26]

In [ ]:
/content/ICDAR-2019-SROIE/task3/src/test.py

In [ ]:
etfo

'IPS SHOP 27 PLAZA AVE, IPSUM (000) 23456789012 CASH RECEIPT ADIPISC 31.25 FEMPOR 18.82 DIPSUM 2.31 DRENIAM 14.30 VULLAMCO 15.27 KABORIS 4.87 ECIDIDUN 54.50 MAGNA 75.00 FALIQUIP 120.50 TOTAL: 459.92 CASH: 370.00 CHANGE: 10.08 THANK YOU FOR SHOPPING \nIPS SHOP 27 PLAZA AVE, IPSUM (000) 23456789012 CASH RECEIPT ADIPISC 31.25 FEMPOR 18.82 DIPSUM 2.31 DRENIAM 14.30 VULLAMCO 15.27 KABORIS 4.87 ECIDIDUN 54.50 MAGNA 75.00 FALIQUIP 120.50 TOTAL: 459.92 CASH: 370.00 CHANGE: 10.08 THANK YOU FOR SHOPPING \nSHOP \nIPS \nPLAZA \nTE \nIPSUM \n(000) \n23456789012 \nRECEIPT \nCASH \n31.25 \nADIPISC \n18.862 \nFEMPOR \nDAK \nDIPSUM \n14.30 \nDRENIAM \n15.27 \nVULLAMCO \n4.87 \nKABORIS \nECIDIDUN \n54.50 \n75.00 \nMAGNA \nFALIQUIP \n120.50 \nTOTAL: \n459.92 \nCASH: \n370.00 \nCHANGE: \n10.08 \nYOU \nTHANK \nSHOPPING \nFOR \n'

In [ ]:
model

ExtractLSTM(
  (embed): Embedding(71, 16)
  (lstm): LSTM(16, 256, num_layers=2, bidirectional=True)
  (linear): Linear(in_features=512, out_features=5, bias=True)
)

In [21]:
!git clone https://github.com/zzzDavid/ICDAR-2019-SROIE

fatal: destination path 'ICDAR-2019-SROIE' already exists and is not an empty directory.


In [23]:
!python /content/ICDAR-2019-SROIE/task3/src/train.py

#0001 | Loss: 1.5909
#0002 | Loss: 1.5171
#0003 | Loss: 1.4186
#0004 | Loss: 1.3366
#0005 | Loss: 1.2737
#0006 | Loss: 1.2811
#0007 | Loss: 1.2209
#0008 | Loss: 1.2084
#0009 | Loss: 1.1846
#0010 | Loss: 1.1395
#0011 | Loss: 1.1543
#0012 | Loss: 1.0804
#0013 | Loss: 1.0519
#0014 | Loss: 0.9554
#0015 | Loss: 0.9756
#0016 | Loss: 0.9359
#0017 | Loss: 0.9800
#0018 | Loss: 0.9598
#0019 | Loss: 0.9456
#0020 | Loss: 0.8701
#0021 | Loss: 0.8340
#0022 | Loss: 0.8971
#0023 | Loss: 0.8901
#0024 | Loss: 0.9314
#0025 | Loss: 0.8042
#0026 | Loss: 0.8071
#0027 | Loss: 0.7836
#0028 | Loss: 0.7136
#0029 | Loss: 0.7131
#0030 | Loss: 0.6634
#0031 | Loss: 0.7773
#0032 | Loss: 0.6445
#0033 | Loss: 0.5715
#0034 | Loss: 0.6524
#0035 | Loss: 0.6463
#0036 | Loss: 0.6253
#0037 | Loss: 0.6527
#0038 | Loss: 0.5841
#0039 | Loss: 0.6526
#0040 | Loss: 0.5320
#0041 | Loss: 0.5954
#0042 | Loss: 0.4487
#0043 | Loss: 0.5153
#0044 | Loss: 0.4774
#0045 | Loss: 0.4651
#0046 | Loss: 0.4354
#0047 | Loss: 0.5022
#0048 | Loss:

In [ ]:
!python /content/ICDAR-2019-SROIE/task3/src/test.py

X00016469670
X00016469671
X51005200931
X51005230605
X51005230616
X51005230621
X51005230648
X51005230657
X51005230659
X51005268275
X51005268408
X51005288570
X51005301666
X51005337867
X51005337877
X51005361906
X51005361908
X51005361912
X51005361923
X51005365187
X51005433518
X51005433543
X51005433548
X51005433556
X51005442322
X51005442334
X51005442343
X51005442366
X51005442375
X51005442382
X51005442388
X51005444040
X51005444041
X51005444044
X51005444046
X51005447841
X51005447842
X51005447844
X51005447851
X51005447859
X51005568855
X51005568866
X51005568885
X51005568887
X51005568889
X51005568890
X51005568892
X51005568894
X51005568895
X51005577191
X51005582745
X51005587261
X51005605287
X51005605295
X51005605332
X51005621482
X51005663274
X51005663300
X51005663307
X51005663309
X51005663310
X51005663323
X51005675099
X51005675103
X51005675104
X51005675914
X51005676534
X51005676535
X51005676537
X51005676542
X51005676548
X51005677327
X51005677333
X51005677336
X51005677337
X510056849111
X5100568494